In [1]:
# for path in sys.path:
#   print(path)

import sys
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

import logging
import argparse # Argument parsing
import math
# from collections import defaultdict
# import concurrent
# from concurrent.futures import ThreadPoolExecutor  # Importing specific executor for clarity
# import time

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Nearest neighbor search
import sklearn
from sklearn.neighbors import BallTree
# Type hints
from typing import Callable, Union, Tuple

# Add your custom path
# sys.path.append("/cache/home/jl2815/tco")

# Custom imports

from GEMS_TCO import orbitmap 
from GEMS_TCO import kernels 
from GEMS_TCO import evaluate
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data



ImportError: cannot import name 'load_data' from 'GEMS_TCO' (/Users/joonwonlee/Documents/GEMS_TCO-1/src/GEMS_TCO/__init__.py)

In [2]:
import sys
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

import GEMS_TCO
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings
from GEMS_TCO import load_data_local_computer

import torch
from collections import defaultdict


In [3]:
lat_lon_resolution = [10,10]
mm_cond_number = 20
years = ['2024']
month_range =[7,8]
idx_for_datamap= [0,8]

params= [20, 8.25, 5.25, 0.2, 0.5, 5]

instance = load_data_local_computer()
map, ord_mm, nns_map= instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

analysis_data_map, aggregated_data = instance.load_working_data_byday( map, ord_mm, nns_map, idx_for_datamap=[0,8])



MLE using full likelihood

MLE using vecchia likelihood

In [4]:
instance = kernels.likelihood_function(smooth=0.5, input_map=analysis_data_map, aggregated_data=aggregated_data,nns_map=nns_map, mm_cond_number=mm_cond_number)

# Define your initial parameters
params = [21.8, 1.09, 1.17, 0.2, .2, 0.5, 1]
params = [52.627, 4, 5.685, 6.77e-2, -4.19e-3, 0.0585, 3.143]  # 50x8 lr=0.01  24.42 1.92, 1.92, 0.001, -0.045, -.237, 3.34
params = [51.79, 3.894, 4.135, -2.08e-2, -7.71e-2, 0.061, 3.5]
params = torch.tensor(params, requires_grad=True)
torch_smooth = torch.tensor(0.5, dtype=torch.float32)


for i in range(10,15):
    mm_cond_number = i 
    print("mm_cond_number: ", mm_cond_number)
    instance = kernels.likelihood_function(smooth=torch_smooth , input_map=analysis_data_map,aggregated_data=aggregated_data, nns_map=nns_map, mm_cond_number=mm_cond_number)

    out0 = instance.full_likelihood(params, aggregated_data[:,:4],aggregated_data[:,2], instance.matern_cov_anisotropy_v05)
    print(out0)
    out0 = instance.vecchia_like_local_computer(params, instance.matern_cov_anisotropy_v05)
    print(out0)

    out0 = instance.vecchia_like_amarel(params, instance.matern_cov_anisotropy_v05)
    print(out0)



mm_cond_number:  10
tensor(2542.7729, grad_fn=<MulBackward0>)
tensor(2640.7498, grad_fn=<AddBackward0>)
tensor(2563.9333, grad_fn=<AddBackward0>)
mm_cond_number:  11
tensor(2542.7729, grad_fn=<MulBackward0>)
tensor(2641.1089, grad_fn=<AddBackward0>)
tensor(2564.2854, grad_fn=<AddBackward0>)
mm_cond_number:  12
tensor(2542.7729, grad_fn=<MulBackward0>)
tensor(2642.1238, grad_fn=<AddBackward0>)
tensor(2565.2751, grad_fn=<AddBackward0>)
mm_cond_number:  13
tensor(2542.7729, grad_fn=<MulBackward0>)
tensor(2641.7437, grad_fn=<AddBackward0>)
tensor(2564.9167, grad_fn=<AddBackward0>)
mm_cond_number:  14
tensor(2542.7729, grad_fn=<MulBackward0>)
tensor(2640.4329, grad_fn=<AddBackward0>)
tensor(2563.6133, grad_fn=<AddBackward0>)


Optimization full likelihood

## 200 x 8

lr 0.001 without scheduler  same as lr, step_size, gamma  0.01 40 0.5  (9.8s)

 Loss: 2549.066650390625, full Parameters: [ 2.48777485e+01  2.05998826e+00  2.16013098e+00  2.20775465e-03
 -7.89414570e-02  1.05411254e-01  3.75236106e+00]

 lr 0.01  step size 40  betas 0.9 , 0.8 gamma 0.9  30 s

  Loss: 2547.1728515625, full Parameters: [ 2.7377291e+01  2.2077193e+00  2.3204505e+00  1.0307773e-03
 -8.0311157e-02  9.8579854e-02  3.6677265e+00]

 lr 0.01  step size 10 betas 0.9 , 0.8 gamma 0.9  30 s
  Loss: 2548.87841796875, full Parameters: [ 2.5092268e+01  2.0689390e+00  2.1694989e+00  2.0285936e-03
 -7.9028614e-02  1.0501490e-01  3.7373385e+00]
Training full likelihood complete.   11.8 sc

 lr 0.01  step size 20 betas 0.9 , 0.8 gamma 0.9  30 s
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]



lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9
 Loss: 2548.18603515625, full Parameters: [ 2.5938652e+01  2.1110108e+00  2.2155209e+00  1.5893303e-03
 -7.9482891e-02  1.0297947e-01  3.6958976e+00]
 21.6

lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 2548.15283203125, full Parameters: [ 2.59814014e+01  2.12175608e+00  2.22699022e+00  1.73025124e-03
 -7.93599486e-02  1.02427535e-01  3.70715070e+00]
 22.9 s

lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9
Loss: 2548.95361328125, full Parameters: [ 2.5118145e+01  1.9827319e+00  2.0768294e+00  1.0898338e-03
 -8.0070712e-02  1.1034889e-01  3.5647078e+00]


## 1250 x 8

1250* 8 55m using constant learning rate 0.0001 
Loss: 14068.798828125, full Parameters: [ 2.46198387e+01  1.61719894e+00  1.76454413e+00  8.55297223e-03
 -1.08275235e-01  1.28809512e-01  2.80795789e+00]

1250* 8 10m 32s
lr 0.01  step size 40 beta 0.9 0.8 gamma 0.9
  Loss: 14068.1953125, full Parameters: [ 2.5030930e+01  1.6107724e+00  1.7573007e+00  8.8407323e-03
 -1.0820019e-01  1.2936097e-01  2.7430327e+00]
Training full likelihood complete.

9m 33s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9
 Loss: 14068.29296875, full Parameters: 
 [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03 -1.0737537e-01  1.2953614e-01 
  2.7420275e+00]
Training full likelihood complete.

#### high resolution data might benefits from larger step size high resolution data often provides 
#### more stable gradients, so larger step size less likely to cause significant fluctuations
14n 41.8s
lr 0.01  step size 10 beta 0.9 0.99 gamma 0.9

FINAL STATE: Epoch 199, 
 Loss: 14068.8828125, full Parameters: 
 [ 2.4707581e+01  1.6489888e+00  1.7993137e+00  8.4043797e-03 -1.0836436e-01  1.2655504e-01  
 2.8416286e+00]

#### beta 0.9 0.99 might be too conservative for high resolution data
13m 44.8s
lr 0.01  step size 20 beta 0.9 0.99 gamma 0.9

 Loss: 14068.318359375, full Parameters: [ 2.4938175e+01  1.6203119e+00  1.7678342e+00  8.6686825e-03
 -1.0813228e-01  1.2845081e-01  2.7731323e+00]


18m
lr 0.01  step size 40 beta 0.9 0.99 gamma 0.9

 Loss: 14067.970703125, full Parameters: [ 2.5205673e+01  1.6159834e+00  1.7630767e+00  8.7957922e-03
 -1.0802399e-01  1.2862283e-01  2.7390635e+00]

9m 52s
lr 0.01  step size 20 beta 0.9 0.8 gamma 0.9

Loss: 14068.29296875, full Parameters: [ 2.4933689e+01  1.6009743e+00  1.7502663e+00  9.2404895e-03
 -1.0737537e-01  1.2953614e-01  2.7420275e+00]
Training full likelihood complete.

Optimization vecchia

In [6]:
params = [24.42, 1.92, 1.92, 0.001, -0.045, 0.237, 3.34]
params = torch.tensor(params, requires_grad=True)

instance = kernels.model_fitting(
    smooth=0.5,
    input_map=analysis_data_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number
)

# optimizer = optim.Adam([params], lr=0.01)  # For Adam
optimizer, scheduler = instance.optimizer_fun( params, lr=0.01, betas=(0.9, 0.99), eps=1e-8, step_size=10, gamma=0.9)  
instance.run_vecc_local(params, optimizer, scheduler,epochs=3000)

Epoch 1, Gradients: [ -1.6978278  -0.9539037  -1.479125    3.4490952  -5.6188226 330.39474
   5.3998504]
 Loss: 2712.965087890625, Parameters: [ 2.442e+01  1.920e+00  1.920e+00  1.000e-03 -4.500e-02  2.370e-01
  3.340e+00]


KeyboardInterrupt: 